In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install sentence_transformers
!pip install kss
!pip install transformers

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sinjy1203/ko-sbert-natenews')

In [4]:
%cd /content/drive/Othercomputers/skt_노트북/ml

/content/drive/Othercomputers/skt_노트북/ml


In [ ]:
# import os
# os.mkdir('ko-sbert-natenews/eval', exist_ok=True)

In [5]:
from finetune import *
from load_dataset import *
news = NateNews()
df = news.load_data()
df = df.reset_index(drop=True)
train_dataset, eval_dataset = dataset(df)
finetuning(train_dataset, eval_dataset, model_path='bongsoo/kpf-sbert-v1.1', output_path='ko-sbert-natenews')

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3001 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3001 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3001 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3001 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3001 [00:00<?, ?it/s]